In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from pyvi import ViTokenizer
from bs4 import BeautifulSoup
from underthesea import word_tokenize
import os

path = "/content/drive/My Drive/analyze_comment_fb/data/"
path_file = os.listdir(path)
# check html in raw text file

# j = 0
# for i in path_file:
# 	j+=1
# 	print(j)
# 	p = path+i
# 	with open(p,"r") as f:
# 		data = f.read()
# 		data = data.split('\n')[1:]
# 		data = ''.join(str(e) for e in data)
# 		if bool(BeautifulSoup(data, "html.parser").find()):
# 			print(p)

#remove stopwords
def remove_stopwords():
	with open('/content/drive/My Drive/analyze_comment_fb/vietnamese-stopwords.txt', 'r') as f:
		list_stopwords = f.read().split('\n')
	cnt = 0
	for i in path_file:
		cnt += 1
		p = path + i
		with open(p, "r") as f:
			data = f.read()
			link, data = data.split('\n')[0], data.split('\n')[1:]
			data = ' '.join(str(e) for e in data)
			data = data.split(' ')
			temp = []
			for w in data:
				if w not in list_stopwords:
					temp.append(w)
			data = ' '.join(str(e) for e in temp)
			preprocessing_file = '/content/drive/My Drive/analyze_comment_fb/preprocessing/' + i
			with open(preprocessing_file, 'w') as write:
				write.write(link + '\n')
				write.write(data)
			print('Saved new file' + preprocessing_file + ' after preprocessing !')
	print("No of saved doc : " + str(cnt))

#Tokenize
def tokenize(doc):
	with open(doc,'r') as f:
		data = f.read()
		link,data = data.split('\n')[0],data.split('\n')[1:]
		data = ' '.join(str(e) for e in data)
		return word_tokenize(data)	




In [0]:
remove_stopwords()

In [0]:
os.remove('/content/drive/My Drive/analyze_comment_fb/preprocessing/9780.txt')
os.remove('/content/drive/My Drive/analyze_comment_fb/preprocessing/9782.txt')
os.remove('/content/drive/My Drive/analyze_comment_fb/preprocessing/9784.txt')


In [13]:
#import preprocessing
import os
import sys
import numpy as np
import keras
import random
from sklearn.model_selection import train_test_split
import pickle
from underthesea import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
path = '/content/drive/My Drive/analyze_comment_fb/preprocessing/'
path_file = os.listdir(path)
path_topic = "/content/drive/My Drive/analyze_comment_fb/topic/"
path_topics = os.listdir(path_topic)
path_topics = [e[:-4] for e in path_topics]
path_topics = sorted(path_topics,key=len,reverse=True)
doc = []
label = []
for i in path_file:
	p = path+i
	with open(p,"r") as f:
		data = f.read()
		link, data = data.split('\n')[0], data.split('\n')[1:]
		link = link[23:link.find('/', 23)]
		check = False
		for tp in range(len(path_topics)):
			t = path_topics[tp]
			if link.find(t) != -1:
				label.append(tp)
				check = True
				break
		if check == False:
			print(p)
			print(link)
		data = ' '.join(str(e) for e in data)
		doc.append(data)
		#print(link+ " "+path_topics[label[-1]])
print(str(len(label))+" "+str(len(doc)))

def diff(list1, list2):
	return list(set(list1).symmetric_difference(set(list2)))

label  = np.array(label)
vectorizer_tfidf = TfidfVectorizer()
vectorizer_tfidf.fit(doc)
X_nmf = vectorizer_tfidf.transform(doc).toarray()
# r = [e for e in range(len(label))]
# train_ = random.sample(r,int(len(label)*0.8))
# X_train , y_train  = np.array([X_nmf[e]  for e in train_]) , np.array([label[e] for e in train_])
# r = diff(r,train_)
# test_ = random.sample(r,int(len(r)/2))
# X_test , y_test  = np.array([X_nmf[e]  for e in test_]) , np.array([label[e] for e in test_])
# r = diff(r,test_)
# X_validate , y_validate = np.array([X_nmf[e]  for e in r]) , np.array([label[e] for e in r])
#sys.exit()
X_train , X_test , y_train , y_test = train_test_split(X_nmf,label,test_size=0.2)
X_train , X_validate , y_train , y_validate = train_test_split(X_train,y_train,test_size=0.1)

model  = keras.models.Sequential()
model.add(keras.layers.Dense(53,activation='softmax'))
model.compile(optimizer='adam',metrics=['acc'],loss='sparse_categorical_crossentropy')
model.fit(X_nmf,label,epochs=50,batch_size=64,validation_data=(X_validate,y_validate))
model.evaluate(X_test,y_test)
# vectorizer_count = CountVectorizer()
# X_lda = vectorizer_count.fit_transform(doc)
# tf_feature_names = vectorizer_count.get_feature_names()
# # Tweak the two parameters below
# number_topics = len(path_topics)
# # Create and fit the LDA model
# lda = LDA(n_components=number_topics, n_jobs=-1)
# lda.fit(X_lda)
# def display_topics(model, feature_names, no_top_words):
# 	for topic_idx, topic in enumerate(model.components_):
# 		print("Topic %d:" % (topic_idx))
# 		print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
#
# no_top_words = 10
# display_topics(lda, tf_feature_names, no_top_words)
# #database = lda.fit_transform(X_lda)
# #np.save('weights',database)
# database = np.load('weights.npy')
# print("Input file directory : ")
# file = input()
# with open(file,"r") as f:
# 	data = f.read()
# 	data = word_tokenize(data)
# 	with open('vietnamese-stopwords.txt','r') as v:
# 		list_stopwords = f.read().split('\n')
# 	res = ""
# 	for w in data:
# 		if w not in list_stopwords:
# 			res+=w
# 	data = [res]
# 	data = vectorizer_count.fit_transform(data)
# 	result = lda.fit_transform(data)
# min = 9999
# topic = ""
# for i in range(len(doc)):
# 	temp = database[i]
# 	dist = np.linalg.norm(temp - result)
# 	if dist<min:
# 		dist = min
# 		topic = path_topics[label[i]]
# print("Your doc topic is : "+topic)
# # check maxlen of doc : 3720
# # maxim = 0
# # for i in path_file:
# # 	p = path+i
# # 	with open(p,'r') as f:
# # 		data = f.read()
# # 		link,data = data.split('\n')[0],data.split('\n')[1:]
# # 		data = ' '.join(str(e) for e in data)
# # 		if maxim < len(word_tokenize(data)):
# # 			maxim = len(word_tokenize(data))
# # print(maxim)
#


12862 12862
Train on 12862 samples, validate on 1029 samples
Epoch 1/50
12862/12862 [==============================] - 3s 264us/step - loss: 3.4731 - acc: 0.4894 - val_loss: 3.0558 - val_acc: 0.5724
Epoch 2/50
12862/12862 [==============================] - 3s 231us/step - loss: 2.7299 - acc: 0.5993 - val_loss: 2.4791 - val_acc: 0.6356
Epoch 3/50
12862/12862 [==============================] - 3s 233us/step - loss: 2.2295 - acc: 0.6554 - val_loss: 2.0629 - val_acc: 0.6812
Epoch 4/50
12862/12862 [==============================] - 3s 231us/step - loss: 1.8672 - acc: 0.7116 - val_loss: 1.7549 - val_acc: 0.7337
Epoch 5/50
12862/12862 [==============================] - 3s 231us/step - loss: 1.5980 - acc: 0.7612 - val_loss: 1.5215 - val_acc: 0.7755
Epoch 6/50
12862/12862 [==============================] - 3s 230us/step - loss: 1.3923 - acc: 0.8033 - val_loss: 1.3399 - val_acc: 0.8134
Epoch 7/50
12862/12862 [==============================] - 3s 237us/step - loss: 1.2313 - acc: 0.8300 - val_loss

[0.15898448344663196, 0.9681305868867488]

In [13]:
!pip install sklearn

In [24]:

data = tokenize('/content/drive/My Drive/analyze_comment_fb/test/test2.txt')
with open('/content/drive/My Drive/analyze_comment_fb/vietnamese-stopwords.txt','r') as f:
    list_stopwords = f.read().split('\n')
res = ""
for w in data:
    if w not in list_stopwords:
        res+=w+" "
data = [res]
X = vectorizer_tfidf.transform(data).toarray()
y = model.predict(X)
print(path_topics[np.argmax(y)])
    

an-ninh-hinh-su
